In [1]:
import src.TX as TX
from src import MACGenerator, UDP_TX, SlidingBook
import numpy as np
import cv2
import time
import socket
import struct
import json


IP = "0.0.0.0"
PORT = 23422
###################################################################################

#### parameters that needs to be exhanged between the sender and the receiver #####
param ={}
                # t1  t2  t3  t4  t5  t6  t7  t8  t9
X = np.array([[ 1,  1,  0,  0,  0,  0,  0,  0,  0], # m1
              [ 1,  1,  0,  0,  0,  0,  0,  0,  0], # m2
              [ 0,  1,  1,  0,  0,  0,  0,  0,  0], # m3
              [ 0,  0,  1,  0,  0,  0,  0,  0,  0], # m4
              [ 0,  0,  1,  1,  0,  0,  0,  0,  0], # m5
              [ 0,  0,  0,  1,  0,  0,  0,  0,  0], # m6
              [ 0,  0,  0,  1,  1,  0,  0,  0,  0], # m7
              [ 0,  0,  0,  0,  1,  0,  0,  0,  0], # m8
              [ 0,  0,  0,  0,  1,  1,  0,  0,  0], # m9
              [ 0,  0,  0,  0,  0,  1,  0,  0,  0], # m10
              [ 0,  0,  0,  0,  0,  1,  1,  0,  0], # m11
              [ 0,  0,  0,  0,  0,  0,  1,  0,  0], # m12
              [ 0,  0,  0,  0,  0,  0,  1,  1,  0], # m13
              [ 0,  0,  0,  0,  0,  0,  0,  1,  0], # m14
              [ 0,  0,  0,  0,  0,  0,  0,  1,  1], # m15
              [ 0,  0,  0,  0,  0,  0,  0,  0,  1], # m16
              [ 0,  0,  0,  0,  0,  0,  0,  0,  1], # m17
              [ 0,  0,  0,  0,  0,  0,  0,  0,  1]]) # m18
 
                # t1  t2  t3  t4  t5  t6  t7  t8  t9
Y = np.array(  [[ 0,  0,  0,  0,  0,  0,  1,  0,  0], # m1
                [ 0,  0,  0,  0,  0,  0,  0,  1,  0], # m2
                [ 1,  0,  0,  0,  0,  0,  0,  0,  0], # m3
                [ 0,  0,  0,  0,  0,  0,  0,  0,  0], # m4
                [ 0,  0,  0,  0,  0,  0,  0,  0,  0], # m5
                [ 0,  1,  0,  0,  0,  0,  0,  0,  0], # m6
                [ 0,  0,  0,  0,  0,  0,  0,  0,  0], # m7
                [ 0,  0,  0,  0,  0,  0,  0,  0,  0], # m8
                [ 0,  0,  1,  0,  0,  0,  0,  0,  0], # m9
                [ 0,  0,  0,  0,  0,  0,  0,  0,  0], # m10
                [ 0,  0,  0,  0,  0,  0,  0,  0,  0], # m11
                [ 0,  0,  0,  1,  0,  0,  0,  0,  0], # m12
                [ 0,  0,  0,  0,  0,  0,  0,  0,  0], # m13
                [ 0,  0,  0,  0,  0,  0,  0,  0,  0], # m14
                [ 0,  0,  0,  0,  1,  0,  0,  0,  0], # m15
                [ 0,  0,  0,  0,  0,  0,  0,  0,  0], # m16
                [ 0,  0,  0,  0,  0,  1,  0,  0,  0], # m17
                [ 0,  0,  0,  0,  0,  0,  0,  0,  1]]) # m18


param['X'] = X.tolist()
param['Y'] = Y.tolist()
param['KEY'] = "key"
param['DIGESTMOD'] = 'sha384'
param['PAYLOAD_SIZE_BYTE']= 64
param['QUALITY'] = 70   # param['QUALITY'] of the JPEG compression
param['ATTACK_PROBABILITY'] = 0.05  # probability of the attack
#########################################################################




# send the parameters to the receiver and the time.time() to synchronize the sender and the receiver
with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    s.connect((IP, PORT))
    s.sendall(json.dumps(param).encode())
    time.sleep(.1)
    data = time.time()
    data = struct.pack('d', data)
    s.send(data)





cap = cv2.VideoCapture(0)  # Capture video from the default camera
# Define the compression parameters for Progressive JPEG
compression_params = [cv2.IMWRITE_JPEG_PROGRESSIVE, 1, cv2.IMWRITE_JPEG_QUALITY, param['QUALITY']]
## counting the number of frames per second
start_time = time.time()
frame_counter = 0
average_tx_size = 0
while True:
    ret, frame = cap.read()

    # Encode the image to JPEG format in memory
    success, encoded_frame = cv2.imencode('.jpg', frame, compression_params)
    data = encoded_frame.tobytes()
    average_tx_size+=len(data)//1000

    #############################  TX  ########################################
    buffer = SlidingBook(num_pages=40, page_size=len(param['X']))
    page_processor = MACGenerator(X = param['X'], Y = param['Y'], secret_key=param['KEY'].encode(), digestmod=param['DIGESTMOD'])
    udp_tx = UDP_TX(IP= IP, PORT= PORT, buffer=buffer, page_processor= page_processor, Payload_Size_Byte = param['PAYLOAD_SIZE_BYTE'])
    pages = udp_tx.segment_to_pages(data)
    udp_tx.transmit(pages, param['ATTACK_PROBABILITY'])
    ###########################################################################

    cv2.imshow('Webcam', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):  # Press 'q' to quit
        break


    # print the frame rate
    frame_counter += 1
    if time.time() - start_time >= 1:
        print("frame per second: ", frame_counter, "fps ,data rate: ", average_tx_size, "KB/s")
        frame_counter = 0
        start_time = time.time()
        average_tx_size = 0
    
cap.release()



[ WARN:0@0.410] global ./modules/videoio/src/cap_gstreamer.cpp (1405) open OpenCV | GStreamer warning: Cannot query video position: status=0, value=-1, duration=-1


frame per second:  27 fps ,data rate:  1049 KB/s
frame per second:  30 fps ,data rate:  1170 KB/s
frame per second:  31 fps ,data rate:  1212 KB/s
frame per second:  30 fps ,data rate:  1173 KB/s
frame per second:  31 fps ,data rate:  1210 KB/s


In [ ]:
import time

time.time()